This is the notebook to scrap the missing data from bitcoin dataset from kaggle.

In [ ]:
import pandas as pd 
import numpy as np
from datetime import datetime
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from time import sleep

In [ ]:
data=pd.read_csv('bitcoin.csv') 

In [ ]:
nn_df=data[data.Open.isnull()]      #create mask for getting rows with null value

In [ ]:
nn_df['dates'] = nn_df['Timestamp'].apply(lambda d: datetime.fromtimestamp(int(d)).strftime('%Y-%m-%d')) #extract the dates for missing rows from unix time

In [ ]:
uniq_dates=list(nn_df['dates'].unique())

The dataset provided on kaggle has been abstracted from [bitcoincharts.com]('https://bitcoincharts.com/charts/bitstampUSD#rg60zczsg2021-04-07zeg2021-06-06ztgSzm1g10zm2g25zv'), this details is been provided in the description section of kaggle dataset. So we need to extract the from data from web site using webscraping and we will use Selenium to extract those data.

<img src='site.png'/>

On the above image , the is link with name `Load raw data`, it contains the value we required . We need to install [Selenium]('https://www.selenium.dev/') and [chrome web driver]('https://sites.google.com/a/chromium.org/chromedriver/') to access chrome for scraping data.

In [ ]:
df=[] ##data would be stored in this list

In [ ]:
for date in uniq_dates:
    driver = webdriver.Chrome(executable_path='')  ## put your path for chrome driver extension    
    url='https://bitcoincharts.com/charts/bitstampUSD#rg60zczsg'+date+'zeg'+date+'ztgSzm1g10zm2g25zv'
    driver.get(url)
    driver.find_element_by_partial_link_text("Load raw data").click()      #click on the `Load raw data`
    sleep(1)
    soup=BeautifulSoup(driver.page_source,'html.parser')
    mydiv=soup.find('table',{'id':'chart_table'})
    rows=mydiv.find('tbody').findAll('tr')
    try:
        for tr in rows:
            if(tr.findAll('td')[1].text!='—'):
                df.append([td.text.strip()  for td in tr.findAll('td')])
    except:
        pass
    sleep(1)
    driver.quit()

This will took around 10-11 hours to extract all required data.

In [ ]:
len(df)

We have extracted 276866 rows of data.

In [ ]:
cp=df.copy()

In [ ]:
cp[0]

In [ ]:
import pickle

In [ ]:
with open("bitcoincharts.txt", "wb") as fp:         ## saving data in text format
    pickle.dump(cp, fp)

In [ ]:
bitcoin=pd.DataFrame(cp,columns=['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume_(BTC)',
       'Volume_(Currency)', 'Weighted_Price'])

In [ ]:
bitcoin

In [ ]:
bitcoin.to_csv('bitcoinchart.csv')

In [ ]:
bitcoin_unix=pd.DataFrame(cp,columns=['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume_(BTC)',
       'Volume_(Currency)', 'Weighted_Price'])

In [ ]:
bitcoin_unix

The date-time has been extracted in timestamp ,but we need to have unix time format in order to append these to previous data.

In [ ]:
bitcoin_unix.info()  

In [ ]:
bitcoin_unix['Timestamp']=(pd.to_datetime(bitcoin_unix.Timestamp)-pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')   ## converting date-time to unix format


In [ ]:
bitcoin_unix.to_csv('bitcoinunix.csv')